# Importing environment variables for google cloud and Neo4J cloud instance


In [22]:
import os

from google.colab import userdata
os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_CLOUD_LOCATION"]=userdata.get('GOOGLE_CLOUD_LOCATION')
os.environ["GOOGLE_CLOUD_PROJECT"]=userdata.get('GOOGLE_CLOUD_PROJECT')
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]=userdata.get('GOOGLE_GENAI_USE_VERTEXAI')
os.environ["NEO4J_PASSWORD"]=userdata.get('NEO4J_PASSWORD')
os.environ["NEO4J_URI"]=userdata.get('NEO4J_URI')
os.environ["NEO4J_USERNAME"]=userdata.get('NEO4J_USERNAME')

# Installing dependencies

In [25]:
%pip install --upgrade langchain langchain-community langchain-google-genai langchain-google-vertexai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

# Loading ANZ (bank) article from wikipedia in Neo4j graph

In [27]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="ANZ (bank)").load()

from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:10])

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-2.5-flash")

from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()

graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/tmp/ipython-input-2250202095.py:17: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo

# Retrieval

In [31]:
from google.colab import auth
auth.authenticate_user()

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/root/.config/gcloud/application_default_credentials.json"

from langchain_community.vectorstores import Neo4jVector

from langchain_google_vertexai import VertexAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    VertexAIEmbeddings(model_name="text-embedding-005"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


GoogleAuthError: 
Unable to authenticate your request.
Depending on your runtime environment, you can complete authentication by:
- if in local JupyterLab instance: `!gcloud auth login` 
- if in Colab:
    -`from google.colab import auth`
    -`auth.authenticate_user()`
- if in service account or other: please follow guidance in https://cloud.google.com/docs/authentication